In [2]:
import pandas as pd
import numpy as np
from gretel_synthetics.timeseries_dgan.dgan import DGAN
from gretel_synthetics.timeseries_dgan.config import DGANConfig
from gretel_synthetics.timeseries_dgan.config import  OutputType
import os
from os import path
from sklearn.preprocessing import MinMaxScaler
from tsl.datasets import AirQuality, MetrLA, PemsBay

dataset_name = 'AirQuality'
df = AirQuality(small=True)

In [3]:
# Create some random training data
df = df.dataframe()[-8736:]
df.columns = df.columns.droplevel('channels')
df_cols = df.columns
scaler = MinMaxScaler()
df = scaler.fit_transform(df)
df = pd.DataFrame(df)
df.columns = df_cols

data = df.values
data = data.reshape(8736//168, 168, 36)

In [4]:
# Train the model
model_path = f'models/{dataset_name}1.model'

if path.exists(model_path):
    model = DGAN.load(model_path)
else:
    model = DGAN(DGANConfig(
        max_sequence_len=168,
        sample_len=24,
        batch_size=13,
        epochs=10,
        apply_feature_scaling=False,
    ))

    model.train_numpy(
        data
    )
    
    model.save(model_path)
# Generate synthetic data
synthetic_df = model.generate_numpy(1000)[1]
# synthetic_df.columns = df.columns

/Users/paulmagos/.pyenv/versions/3.11.9/envs/Thesis/lib/python3.11/site-packages/gretel_synthetics/timeseries_dgan/dgan.py:996: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
 

In [7]:
synthetic_df.shape

(1000, 168, 36)

In [16]:
# synthetic_df.to_csv(f'{os.getcwd()}/../../Datasets/GeneratedDatasets/{dataset_name}/syntetic{dataset_name}DGAN.csv', index=False)